In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dense, Dropout, Add, 
                                     MultiHeadAttention, LayerNormalization, GlobalAveragePooling2D)
from tensorflow.keras import Model
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim


# Image Preprocessing

In [4]:
import os
import glob
import numpy as np
import tensorflow as tf
from skimage.io import imread
from skimage import img_as_float
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess images from a given folder
def load_images_from_folder(folder, size=(128, 128)):
    images = []
    for filename in glob.glob(os.path.join(folder, '*.png')):
        img = imread(filename)
        img = img_as_float(img)
        img_resized = tf.image.resize(img, size).numpy()
        images.append(img_resized)
    return np.array(images)

# Data preprocessing and augmentation pipeline
def preprocess_and_augment_data(images, augment=False):
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.01,
            zoom_range=[0.9, 1.25],
            horizontal_flip=True,
            fill_mode='reflect'
        )
        augmented_images = datagen.flow(images, batch_size=len(images), shuffle=False)
        augmented_images = next(augmented_images)  # Fetch the augmented images
        return augmented_images
    return images

# Function to load datasets from base folder (for both training and testing)
def load_and_preprocess_datasets(base_folder, size=(128, 128), augment=False):
    rainy_images_train, clear_images_train = [], []
    rainy_images_test, clear_images_test = [], []  # Corrected initialization

    datasets = ['Rain200L', 'Rain200H']  # Add more datasets if necessary

    for dataset in datasets:
        train_folder = os.path.join(base_folder, dataset, 'train')
        test_folder = os.path.join(base_folder, dataset, 'test')

        # Load training data
        rainy_train_folder = os.path.join(train_folder, 'input')
        clear_train_folder = os.path.join(train_folder, 'target')
        rainy_images_train.extend(load_images_from_folder(rainy_train_folder, size))
        clear_images_train.extend(load_images_from_folder(clear_train_folder, size))

        # Load testing data
        rainy_test_folder = os.path.join(test_folder, 'input')
        clear_test_folder = os.path.join(test_folder, 'target')
        rainy_images_test.extend(load_images_from_folder(rainy_test_folder, size))
        clear_images_test.extend(load_images_from_folder(clear_test_folder, size))

    # Convert lists to numpy arrays
    rainy_images_train = np.array(rainy_images_train)
    clear_images_train = np.array(clear_images_train)
    rainy_images_test = np.array(rainy_images_test)
    clear_images_test = np.array(clear_images_test)

    # Apply data augmentation to training data (if augment is True)
    if augment:
        rainy_images_train = preprocess_and_augment_data(rainy_images_train, augment=True)
        clear_images_train = preprocess_and_augment_data(clear_images_train, augment=True)

    return (rainy_images_train, clear_images_train), (rainy_images_test, clear_images_test)

# Example usage
base_folder = '/kaggle/input/derainingdata/RainData'
(train_rainy, train_clear), (test_rainy, test_clear) = load_and_preprocess_datasets(base_folder, size=(128, 128), augment=True)

print(f"Training Rainy Images Shape: {train_rainy.shape}")
print(f"Training Clear Images Shape: {train_clear.shape}")
print(f"Testing Rainy Images Shape: {test_rainy.shape}")
print(f"Testing Clear Images Shape: {test_clear.shape}")


Training Rainy Images Shape: (3600, 128, 128, 3)
Training Clear Images Shape: (3600, 128, 128, 3)
Testing Rainy Images Shape: (400, 128, 128, 3)
Testing Clear Images Shape: (400, 128, 128, 3)


# Advanced Dynamic Pyramid Model

In [66]:
import tensorflow as tf
from keras.layers import Input, Conv2D, Add, Lambda, LayerNormalization, MultiHeadAttention, Dense, Layer, UpSampling2D
from tensorflow.keras.models import Model

# Advanced Dynamic Pyramid Model
def advanced_dynamic_pyramid_model(input_shape, num_scales=3):
    inputs = Input(shape=input_shape)
    pyramid_features = []

    for scale in range(num_scales):
        scale_factor = 2 ** scale
        
        # Resizing each input for different scales
        x_scaled = Lambda(lambda x: tf.image.resize(x, 
                            [input_shape[0] // scale_factor, input_shape[1] // scale_factor]))(inputs)
        
        # Dynamic kernel size based on scale
        kernel_size = 3 + scale
        x_conv = Conv2D(64 * scale_factor, (kernel_size, kernel_size), activation='relu', padding='same')(x_scaled)
        
        # Residual connection
        x_scaled_conv = Conv2D(64 * scale_factor, (1, 1), padding='same')(x_scaled)
        x_conv = Add()([x_conv, x_scaled_conv])
        
        # Resize back to original input size before concatenation
        x_resized = Lambda(lambda x: tf.image.resize(x, 
                              [input_shape[0], input_shape[1]]))(x_conv)
        
        pyramid_features.append(x_resized)

    # Concatenate all pyramid features along channel axis
    fused_features = Lambda(
        lambda x: tf.concat(x, axis=-1),
    )(pyramid_features)
    
    return inputs, fused_features

# Custom Layer: Window Partition Layer
class WindowPartitionLayer(Layer):
    def __init__(self, window_size):
        super(WindowPartitionLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size, height, width, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        windowed = tf.image.extract_patches(
            inputs,
            sizes=[1, self.window_size, self.window_size, 1],
            strides=[1, self.window_size, self.window_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        windowed = tf.reshape(windowed, (batch_size, -1, self.window_size * self.window_size, channels))
        return windowed

# Custom Layer: Patch Merge Layer
class PatchMergeLayer(Layer):
    def __init__(self, window_size):
        super(PatchMergeLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        batch_size, num_windows, flattened_window_size, channels = tf.shape(inputs)[0], tf.shape(inputs)[1], tf.shape(inputs)[2], tf.shape(inputs)[3]
        merged = tf.reshape(inputs, (batch_size, int(num_windows**0.5), int(num_windows**0.5), self.window_size, self.window_size, channels))
        merged = tf.transpose(merged, perm=[0, 1, 3, 2, 4, 5])
        merged = tf.reshape(merged, (batch_size, merged.shape[1] * self.window_size, merged.shape[3] * self.window_size, channels))
        return merged, self.window_size * 2

# Modified Swin Transformer Block
class ModifiedSwinTransformerBlock(Layer):
    def __init__(self, initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6):
        super(ModifiedSwinTransformerBlock, self).__init__()
        self.initial_window_size = initial_window_size
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.num_recursions = num_recursions

    def call(self, fused_features):
        original_shape = tf.shape(fused_features)
        return self.recursive_block(fused_features, self.initial_window_size, 2, self.num_recursions, original_shape)

    def recursive_block(self, x, window_size, iteration, recursion, original_shape):
        if recursion == 0:
            return x
        
        for _ in range(iteration):
            # Partition the window
            partition_layer = WindowPartitionLayer(window_size)
            windows = partition_layer(x)

            window_shape = (tf.shape(windows)[-2], tf.shape(windows)[-1])
            windows_reshaped = tf.reshape(windows, (-1, window_shape[0] * window_shape[1], tf.shape(x)[-1]))

            # Ensure input is reshaped correctly for LayerNormalization
            x_norm = LayerNormalization(axis=-1)(windows_reshaped)

            attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(x_norm, x_norm)
            attention_scores = tf.reduce_mean(tf.abs(attention), axis=-1, keepdims=True)
            threshold = 0.1
            attention = tf.where(attention_scores > threshold, attention, tf.zeros_like(attention))

            x_add = Add()([windows_reshaped, attention])
            x_reconstructed = self.window_reverse(x_add, original_shape)

            # Apply LayerNormalization and FFN
            x_norm_ffn = LayerNormalization(axis=-1)(x_reconstructed)
            x_ffn = Dense(128, activation='relu')(x_norm_ffn)
            x_ffn_out = Dense(tf.shape(x)[-1])(x_ffn)

            x_out = Add()([x_reconstructed, x_ffn_out])
            
            # Merge the patches and continue recursion
            patch_merge_layer = PatchMergeLayer(window_size)
            x_out, window_size = patch_merge_layer(x_out)

        return self.recursive_block(x_out, window_size, iteration, recursion - 1, original_shape)

    def window_reverse(self, x, original_shape):
        batch_size = tf.shape(x)[0]
        num_windows = tf.shape(x)[1] // (self.initial_window_size * self.initial_window_size)
        x = tf.reshape(x, (batch_size, num_windows, self.initial_window_size, self.initial_window_size, -1))
        return tf.reshape(x, (batch_size, original_shape[1], original_shape[2], -1))

# Image Restoration and Enhancement Block
def image_restoration_and_enhancement(x_out, num_classes=3):
    x = LayerNormalization()(x_out)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = LayerNormalization()(x)
    outputs = Conv2D(num_classes, (3, 3), padding='same', activation='tanh')(x)
    return outputs

# Full Model Construction
def build_full_model(input_shape):
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)
    transformer_block = ModifiedSwinTransformerBlock(initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6)
    transformer_output = transformer_block(pyramid_features)
    outputs = image_restoration_and_enhancement(x_out=transformer_output, num_classes=3)
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

# Compile the model
input_shape = (256, 256, 3)  # Example input shape
model = build_full_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Summary of the model
model.summary()


RuntimeError: Exception encountered when calling ModifiedSwinTransformerBlock.call().

[1mCould not automatically infer the output shape / dtype of 'modified_swin_transformer_block_7' (of type ModifiedSwinTransformerBlock). Either the `ModifiedSwinTransformerBlock.call()` method is incorrect, or you need to implement the `ModifiedSwinTransformerBlock.compute_output_spec() / compute_output_shape()` method. Error encountered:

Shapes used to initialize variables must be fully-defined (no `None` dimensions). Received: shape=(None,) for variable path='modified_swin_transformer_block_7/layer_normalization_16/gamma'[0m

Arguments received by ModifiedSwinTransformerBlock.call():
  • args=('<KerasTensor shape=(None, 256, 256, 448), dtype=float32, sparse=False, name=keras_tensor_637>',)
  • kwargs=<class 'inspect._empty'>

# Window Partition and Reverse Functions

In [48]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class WindowPartitionLayer(Layer):
    def __init__(self, window_size):
        super(WindowPartitionLayer, self).__init__()
        self.window_size = window_size

    def call(self, inputs):
        patches = tf.image.extract_patches(images=inputs,
                                           sizes=[1, self.window_size, self.window_size, 1],
                                           strides=[1, self.window_size, self.window_size, 1],
                                           rates=[1, 1, 1, 1],
                                           padding='VALID')
        return patches

def window_partition(x, window_size):
    partition_layer = WindowPartitionLayer(window_size)
    return partition_layer(x)


# Patch Merging Function

In [41]:
# Patch Merging Function
def patch_merge(x, window_size):
    """Merge adjacent patches to create a larger feature window."""
    partition_layer = WindowPartitionLayer(window_size)
    patches = partition_layer(x)

    # Calculate the new merged patch (by averaging or pooling)
    merged_patches = tf.reduce_mean(patches, axis=-1, keepdims=True)

    # Reshape into the original window size
    new_window_size = window_size * 2
    merged = window_reverse(merged_patches, new_window_size, x.shape)

    return merged, new_window_size

# Modified Swin Transformer Block with Recursion and Iteration

In [52]:
from tensorflow.keras.layers import Layer

def modified_swin_transformer_block(fused_features, initial_window_size, num_heads, key_dim, num_recursions):
    # Define a recursive block as a Keras layer
    class RecursiveBlock(Layer):
        def __init__(self, window_size, num_heads, key_dim, num_recursions):
            super().__init__()
            self.window_size = window_size
            self.num_heads = num_heads
            self.key_dim = key_dim
            self.num_recursions = num_recursions

        def call(self, x):
            if self.num_recursions <= 0:
                return x
            
            # Partition the input into windows
            windows = window_partition(x, self.window_size)
            window_shape = (windows.shape[-2], windows.shape[-1])
            windows_reshaped = tf.reshape(windows, (-1, window_shape[0] * window_shape[1], x.shape[-1]))

            # Layer Normalization before Attention
            x_norm = LayerNormalization()(windows_reshaped)
            attention = MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(x_norm, x_norm)

            # Apply reverse window operation
            x_out = window_reverse(attention, window_shape, x.shape)
            x_out, window_size = patch_merge(x_out, window_size)

            # Recur on the output
            return RecursiveBlock(window_size, self.num_heads, self.key_dim, self.num_recursions - 1)(x_out)

    # Instantiate and call the recursive block
    recursive_layer = RecursiveBlock(initial_window_size, num_heads, key_dim, num_recursions)
    return recursive_layer(fused_features)


# Image Restoration and Enhancement

In [50]:
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, Conv2D, UpSampling2D

def image_restoration_and_enhancement(x_out, num_classes=3):
    x = LayerNormalization()(x_out)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = LayerNormalization()(x)
    outputs = Conv2D(num_classes, (3, 3), padding='same', activation='tanh')(x)
    return outputs


# Construct and Compile the Model

In [53]:
from tensorflow.keras.models import Model

def build_full_model(input_shape):
    inputs = Input(shape=input_shape)
    inputs, pyramid_features = advanced_dynamic_pyramid_model(input_shape=input_shape, num_scales=3)
    transformer_output = modified_swin_transformer_block(fused_features=pyramid_features, 
                                                         initial_window_size=4, num_heads=4, key_dim=64, num_recursions=6)
    outputs = image_restoration_and_enhancement(x_out=transformer_output, num_classes=3)
    model = Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (256, 256, 3)
model = build_full_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()


TypeError: Exception encountered when calling RecursiveBlock.call().

[1munsupported operand type(s) for //: 'int' and 'tuple'[0m

Arguments received by RecursiveBlock.call():
  • args=('<KerasTensor shape=(None, 256, 256, 448), dtype=float32, sparse=False, name=keras_tensor_239>',)
  • kwargs=<class 'inspect._empty'>

# Model Training

In [ ]:
# Import necessary libraries for training
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

# Set up input shape and define the model
input_shape = (256, 256, 3)
model = build_full_model(input_shape)

# Compile the model with Adam optimizer and MSE loss
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Define callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define training parameters
epochs = 50
batch_size = 16

# Train the model
history = model.fit(train_data, train_labels,
                    validation_data=(val_data, val_labels),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint, early_stopping])

# Save the final model
model.save('final_model.h5')


# Model Evaluation

In [ ]:
# Import necessary libraries for evaluation
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)

# Print the test results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


# PSNR and SSIM Calculation

In [ ]:
# Import necessary libraries for PSNR and SSIM calculations
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np

# Function to calculate PSNR
def calculate_psnr(ground_truth, prediction):
    return peak_signal_noise_ratio(ground_truth, prediction, data_range=1.0)

# Function to calculate SSIM
def calculate_ssim(ground_truth, prediction):
    return structural_similarity(ground_truth, prediction, multichannel=True, data_range=1.0)

# Use the trained model to predict on the test data
predictions = model.predict(test_data)

# Initialize lists to store PSNR and SSIM values
psnr_list = []
ssim_list = []

# Loop through each test image and calculate PSNR and SSIM
for i in range(len(test_data)):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    psnr = calculate_psnr(gt_image, pred_image)
    ssim = calculate_ssim(gt_image, pred_image)
    
    psnr_list.append(psnr)
    ssim_list.append(ssim)

# Calculate average PSNR and SSIM
average_psnr = np.mean(psnr_list)
average_ssim = np.mean(ssim_list)

# Print the results
print(f"Average PSNR: {average_psnr:.4f}")
print(f"Average SSIM: {average_ssim:.4f}")


# Prediction and Evaluation on Test Images

In [ ]:
# Import necessary libraries for visualization
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('best_model.h5')

# Predict on test data
predictions = model.predict(test_data)

# Visualize the results
num_images_to_display = 3

for i in range(num_images_to_display):
    gt_image = test_labels[i]
    pred_image = predictions[i]
    
    # Display ground truth and prediction side by side
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.title("Ground Truth")
    plt.imshow((gt_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.subplot(1, 2, 2)
    plt.title("Predicted")
    plt.imshow((pred_image * 255).astype(np.uint8))  # Convert to range [0, 255] for display
    
    plt.show()


# PSNR and SSIM Evaluation Across Test Set

In [ ]:
# PSNR and SSIM Evaluation Across the Test Set
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Function to evaluate PSNR and SSIM for the entire test set
def evaluate_test_set_psnr_ssim(test_data, test_labels, predictions):
    psnr_values = []
    ssim_values = []

    for i in range(len(test_data)):
        gt_image = test_labels[i]
        pred_image = predictions[i]
        
        psnr = peak_signal_noise_ratio(gt_image, pred_image, data_range=1.0)
        ssim = structural_similarity(gt_image, pred_image, multichannel=True, data_range=1.0)
        
        psnr_values.append(psnr)
        ssim_values.append(ssim)

    # Compute the average PSNR and SSIM
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)

    print(f"Average PSNR: {avg_psnr:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")
    
    return avg_psnr, avg_ssim

# Get predictions from the model
predictions = model.predict(test_data)

# Evaluate PSNR and SSIM on the test set
avg_psnr, avg_ssim = evaluate_test_set_psnr_ssim(test_data, test_labels, predictions)


# Visualization of Training History

In [ ]:
# Import libraries for visualization
import matplotlib.pyplot as plt

# Plot the training and validation loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot the training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.show()
